# ETUDE DU VIN

## Intro
Nous avons choisit de récupérer une base de données sur les vins du monde. Cette base contient de nombreuses informations telles que le nom du vin, la région d'où il vient, son prix, sa qualité, etc...
Un des objectifs du projet est de savoir si les meilleurs vins sont les plus chèrs. En effet, on peut se demander si le prix est bien fonction de sa qualité. Le prix des vins pourrait etre uniquement lié à la renommée de ceux-ci. 
On propose alors de faire une régression entre le prix et la qualité du vin. 

### I. Régression prix/qualité 

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# Load the data

path='data.csv'

print('debug')
df = pd.read_csv(path)

print(df.keys())

# Extract X (price) and y (points)
X = df['price'].values
y = df['points'].values

# Remove any rows where price is NaN 
valid_indices = ~np.isnan(X) & ~np.isnan(y)
x = X[valid_indices]
y = y[valid_indices]

log_x = np.log(x)
coefficients = np.polyfit(log_x, y, 1) # returns [b, a]

b = coefficients[0]
a = coefficients[1]

y_fit = a + b * np.log(x)

# Plot the data and regression line
plt.scatter(x, y, color='blue', label='Data')
plt.scatter(x, y_fit, color='red', label='Regression Line')
plt.xlabel('Price ($)')
plt.ylabel('Points')

'''
plt.xlim(left=0)   # set x-axis minimum to 0
plt.ylim(bottom=0) # set y-axis minimum to 0
plt.ylim(top=100)  # set y-axis maximum to 100

'''

plt.title('Linear Regression: Points vs. Price')
plt.legend()
plt.show()

print('finished')



